In [61]:
# Import all necessary modules
import numpy as np
import matplotlib.pyplot as plt
import glob, json
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import r2_score
from sklearn.metrics import confusion_matrix

from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report

In [4]:
# Load file names and labels for the processed data
data_folder_prefix = "250x250/Seg_Featured_"

with open("250x250/data_labels.json", 'r') as json_file:
    label_dict = json.load(json_file)


file_names = list(label_dict.keys())
print(len(file_names))

labels = []
for fl in file_names:
    labels.append(label_dict[fl])
print(len(labels))
files = [data_folder_prefix+x+".npy" for x in file_names]

743
743


In [5]:
# Revised 

# files = glob.glob("Seg_Featured_Data/*.npy")
max_len = 250*250+7
num_padded = 0
flattened_test_load = np.zeros((max_len, ))
print(flattened_test_load.shape)

for fl in files:
    test_load = np.load(fl, None, allow_pickle=True)
    y,x,_ = test_load.shape

    # Ignore depth 0
    # Keep all Depth 1
    #  if the x,y is less than 250x50 pad w 0s
    layers_depth = test_load[:,:,1]
    # print(layers_depth.shape)
    if x < 250:
        layers_depth = np.hstack((layers_depth, np.zeros((y, y-x))))
        num_padded +=1
    if y < 250:
        layers_depth = np.hstack((layers_depth, np.zeros((250-y, 250))))
    layers_depth = layers_depth.flatten()
    # print(layers_depth.shape)


    # Get the metrics from the other depths
    vl, ct = np.unique(test_load[:,:,5], return_counts=True)
    if len(ct)>1:
        num_bp = ct[1]
    else:
        num_bp = 0

    # print("conected components (mean, std)", np.mean(test_load[:y//10-1,:x//10-1,2]), np.std(test_load[:y//10-1,:x//10-1,2]))
    # print("orientaiton (mean, std)", np.mean(test_load[:y//10-1,:x//10-1,3]), np.std(test_load[:y//10-1,:x//10-1,3]))
    # print("mean orientaiton (mean, std)", np.mean(test_load[:250//10-1,:250//10-1,4]), np.std(test_load[:250//10-1,:250//10-1,4]))
    # print("num breakpoints",num_bp)
    # print("dist map (mean, std)", np.mean(test_load[:,:,6]), np.std(test_load[:,:,6]))

    data= np.append(layers_depth, [np.mean(test_load[:y//10-1,:x//10-1,2]), np.std(test_load[:y//10-1,:x//10-1,2]),
                                    np.mean(test_load[:y//10-1,:x//10-1,3]), np.std(test_load[:y//10-1,:x//10-1,3]),
                                    num_bp,
                                    np.mean(test_load[:,:,6]), np.std(test_load[:,:,6])])

    # print(data.shape)
    flattened_test_load = np.vstack((flattened_test_load, data))

print(flattened_test_load.shape,flattened_test_load[0].shape, "num padded:", num_padded)

flattened_data = flattened_test_load[1:,:]
flattened_data.shape, len(labels)


(62507,)
(744, 62507) (62507,) num padded: 106


((743, 62507), 743)

In [62]:
X_train, X_test, y_train, y_test = train_test_split(flattened_data, labels, test_size=0.2)

In [63]:
print(X_train.shape)
print(X_test.shape)
print(len(y_train))
print(len(y_test))

(594, 62507)
(149, 62507)
594
149


In [64]:
def mse(y_true, y_pred):
    return np.average(np.square(np.subtract(y_true, y_pred)))

In [65]:
def rmse(y_true, y_pred):
    return np.sqrt(np.average(np.square(np.subtract(y_true, y_pred))))

In [66]:
def mae(y_true, y_pred):
    return np.average(np.absolute(np.subtract(y_true, y_pred)))

In [67]:
# Build a LinearRegression model and train it on (X_train, y_train)
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

LinearRegression()

In [68]:
# Test lin_reg on X_test
y_pred_lin_reg = lin_reg.predict(X_test)

mae_lin_reg = mean_absolute_error(y_test, y_pred_lin_reg)
print('Linear Regression MAE', mae_lin_reg)

mse_lin_reg = mean_squared_error(y_test, y_pred_lin_reg)
print('Linear Regression MSE', mse_lin_reg)

Linear Regression MAE 1.0978241448600956
Linear Regression MSE 1.8234355376908475


In [69]:
# Build a SVR (SVM Regression) model and train it on (X_train, y_train), kernel should be 'linear'
svr = SVR(kernel='linear')
svr.fit(X_train, y_train)

SVR(kernel='linear')

In [70]:
# Test svr
y_pred_svr = svr.predict(X_test)

In [71]:
# Test lin_reg on X_test
y_pred_svr = svr.predict(X_test)

mae_srv = mean_absolute_error(y_test, y_pred_svr)
print('SRV MAE', mae_srv)

mse_srv = mean_squared_error(y_test, y_pred_svr)
print('SRV MSE', mse_srv)

SRV MAE 1.0761748009957635
SRV MSE 1.7460572011986784


In [72]:
# Build a xgb regression model and train it on (X_train, y_train)
xgb_reg = XGBRegressor(objective ='reg:squarederror')
xgb_reg.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [73]:
# Test xgb_reg
y_pred_xgb = xgb_reg.predict(X_test)


mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
print('SRV MAE', mae_xgb)

mse_xgb = mean_squared_error(y_test, y_pred_xgb)
print('SRV MSE', mse_xgb)

SRV MAE 0.43659789232348073
SRV MSE 0.36365495153061933


In [84]:
y_pred_xgb10 = np.rint(y_pred_xgb)

In [85]:
y_test10 = np.rint(y_test)

In [86]:
confusion_matrix(y_test10, y_pred_xgb10)

array([[ 6,  1,  1,  1,  0],
       [ 2,  2,  0,  0,  0],
       [ 0,  0,  0,  1,  2],
       [ 0,  0,  1,  7, 22],
       [ 0,  0,  0, 21, 82]], dtype=int64)

In [87]:
balanced_acc = balanced_accuracy_score(y_test10, y_pred_xgb10)
print(balanced_acc)

0.4392233009708738


In [88]:
print(classification_report(y_test10, y_pred_xgb10))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00         1
         1.0       1.00      1.00      1.00         1
         2.0       0.00      0.00      0.00         1
         3.0       0.33      1.00      0.50         1
         4.0       1.00      0.83      0.91         6

    accuracy                           0.80        10
   macro avg       0.67      0.77      0.68        10
weighted avg       0.83      0.80      0.80        10



C:\Users\lukez\anaconda3\envs\ComputerVision\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\lukez\anaconda3\envs\ComputerVision\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\lukez\anaconda3\envs\ComputerVision\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

In [89]:
print(y_test10[0:10])

[4. 4. 4. 2. 1. 4. 4. 4. 0. 3.]


In [90]:
print(y_pred_xgb10[0:10])

[ 4.  4.  4.  3.  1.  4.  4.  3. -0.  3.]


In [156]:
# Build a SVR (SVM Regression) model and train it on (X_train, y_train), kernel should be 'linear'
svrL = SVR(kernel='linear')
svrL.fit(X_train1, y_train1)

# Test lin_reg on X_test
y_pred_svrL = svrL.predict(X_test1)

mae_svrL = mean_absolute_error(y_test, np.multiply(y_pred_svrL, 4))
print('SRV MAE', mae_svrL)

mse_svrL = mean_squared_error(y_test, np.multiply(y_pred_svrL, 4))
print('SRV MSE', mse_svrL)

SRV MAE 1.1745408068720213
SRV MSE 2.1296426691340837


In [157]:
max_svrL = np.max(y_pred_svrL)

In [167]:
y_pred_svr10 = np.rint(np.multiply(y_pred_svrL, 4))
y_test10 = np.rint(y_test)
print(classification_report(y_test10, y_pred_svr10))

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         9
         1.0       0.00      0.00      0.00         4
         2.0       0.00      0.00      0.00         3
         3.0       0.26      0.40      0.31        30
         4.0       0.69      0.23      0.35       103
         5.0       0.00      0.00      0.00         0
         6.0       0.00      0.00      0.00         0
         7.0       0.00      0.00      0.00         0

    accuracy                           0.24       149
   macro avg       0.12      0.08      0.08       149
weighted avg       0.53      0.24      0.30       149



C:\Users\lukez\anaconda3\envs\ComputerVision\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\lukez\anaconda3\envs\ComputerVision\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\lukez\anaconda3\envs\ComputerVision\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\

In [159]:
# Build a SVR (SVM Regression) model and train it on (X_train, y_train), kernel should be 'linear'
svrR = SVR(kernel='rbf')
svrR.fit(X_train1, y_train1)

# Test lin_reg on X_test
y_pred_svrR = svrR.predict(X_test1)

mae_svrR = mean_absolute_error(y_test, np.multiply(y_pred_svrR, 4))
print('SRV MAE', mae_svrR)

mse_svrR = mean_squared_error(y_test, np.multiply(y_pred_svrR, 4))
print('SRV MSE', mse_svrR)

SRV MAE 0.7670755185421129
SRV MSE 1.1512741356769691


In [165]:
y_pred_svr10 = np.rint(np.multiply(y_pred_svrR, 4))
y_test10 = np.rint(y_test)
print(classification_report(y_test10, y_pred_svr10))

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         9
         1.0       0.00      0.00      0.00         4
         2.0       0.00      0.00      0.00         3
         3.0       0.15      0.37      0.21        30
         4.0       0.67      0.50      0.57       103

    accuracy                           0.42       149
   macro avg       0.16      0.17      0.16       149
weighted avg       0.49      0.42      0.44       149



C:\Users\lukez\anaconda3\envs\ComputerVision\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\lukez\anaconda3\envs\ComputerVision\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\lukez\anaconda3\envs\ComputerVision\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

In [160]:
# Build a SVR (SVM Regression) model and train it on (X_train, y_train), kernel should be 'linear'
svr3 = SVR(kernel="poly", degree=3)
svr3.fit(X_train1, y_train1)

# Test lin_reg on X_test
y_pred_svr3 = svr3.predict(X_test1)

mae_svr3 = mean_absolute_error(y_test, np.multiply(y_pred_svr3, 4))
print('SRV MAE', mae_svr3)

mse_svr3 = mean_squared_error(y_test, np.multiply(y_pred_svr3, 4))
print('SRV MSE', mse_svr3)

SRV MAE 0.738059651356792
SRV MSE 1.2540495505330327


In [166]:
y_pred_svr10 = np.rint(np.multiply(y_pred_svr3, 4))
y_test10 = np.rint(y_test)
print(classification_report(y_test10, y_pred_svr10))

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         9
         1.0       0.00      0.00      0.00         4
         2.0       0.00      0.00      0.00         3
         3.0       0.00      0.00      0.00        30
         4.0       0.68      0.97      0.80       103
         7.0       0.00      0.00      0.00         0

    accuracy                           0.67       149
   macro avg       0.11      0.16      0.13       149
weighted avg       0.47      0.67      0.56       149



C:\Users\lukez\anaconda3\envs\ComputerVision\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\lukez\anaconda3\envs\ComputerVision\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\lukez\anaconda3\envs\ComputerVision\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\

In [161]:
# Build a SVR (SVM Regression) model and train it on (X_train, y_train), kernel should be 'linear'
svr4 = SVR(kernel="poly", degree=4)
svr4.fit(X_train1, y_train1)

# Test lin_reg on X_test
y_pred_svr4 = svr4.predict(X_test1)

mae_svr4 = mean_absolute_error(y_test, np.multiply(y_pred_svr4, 4))
print('SRV MAE', mae_svr4)

mse_svr4 = mean_squared_error(y_test1, np.multiply(y_pred_svr4, 4))
print('SRV MSE', mse_svr4)

SRV MAE 1.2589729666140712
SRV MSE 54.15576282059471


In [162]:
# Build a SVR (SVM Regression) model and train it on (X_train, y_train), kernel should be 'linear'
svrS = SVR(kernel="sigmoid")
svrS.fit(X_train1, y_train1)

# Test lin_reg on X_test
y_pred_svrS = svrS.predict(X_test1)

mae_svrS = mean_absolute_error(y_test, np.multiply(y_pred_svr5, 4))
print('SRV MAE', mae_svrS)

mse_svrS = mean_squared_error(y_test, np.multiply(y_pred_svr5, 4))
print('SRV MSE', mse_svrS)

NameError: name 'y_pred_svr5' is not defined

In [110]:
# Build a xgb regression model and train it on (X_train, y_train)
xgb_reg = XGBRegressor(objective ='reg:squarederror')
xgb_reg.fit(X_train, y_train)

# Test xgb_reg
y_pred_xgb = xgb_reg.predict(X_test)


mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
print('SRV MAE', mae_xgb)

mse_xgb = mean_squared_error(y_test, y_pred_xgb)
print('SRV MSE', mse_xgb)

SRV MAE 0.43659789232348073
SRV MSE 0.36365495153061933


In [111]:
# Build a xgb regression model and train it on (X_train, y_train)
xgb_reg1 = XGBRegressor(objective ='reg:squarederror', scale_pos_weight = .2)
xgb_reg1.fit(X_train, y_train)

# Test xgb_reg
y_pred_xgb1 = xgb_reg1.predict(X_test)


mae_xgb1 = mean_absolute_error(y_test, y_pred_xgb1)
print('SRV MAE', mae_xgb1)

mse_xgb1 = mean_squared_error(y_test, y_pred_xgb1)
print('SRV MSE', mse_xgb1)

SRV MAE 0.436902011732177
SRV MSE 0.38044545042611766


In [112]:
# Build a xgb regression model and train it on (X_train, y_train)
xgb_reg2 = XGBRegressor(objective ='reg:squarederror', scale_pos_weight = .5)
xgb_reg2.fit(X_train, y_train)

# Test xgb_reg
y_pred_xgb2 = xgb_reg2.predict(X_test)


mae_xgb2 = mean_absolute_error(y_test, y_pred_xgb2)
print('SRV MAE', mae_xgb2)

mse_xgb2 = mean_squared_error(y_test, y_pred_xgb2)
print('SRV MSE', mse_xgb2)

SRV MAE 0.4214308082669873
SRV MSE 0.3367802788569612


In [168]:
y_pred_xgb10 = np.rint(y_pred_xgb2)
y_test10 = np.rint(y_test)
print(classification_report(y_test10, y_pred_xgb10))

              precision    recall  f1-score   support

         0.0       0.75      0.67      0.71         9
         1.0       0.67      0.50      0.57         4
         2.0       0.00      0.00      0.00         3
         3.0       0.16      0.17      0.16        30
         4.0       0.75      0.77      0.76       103

    accuracy                           0.62       149
   macro avg       0.46      0.42      0.44       149
weighted avg       0.61      0.62      0.61       149



In [113]:
# Build a xgb regression model and train it on (X_train, y_train)
xgb_reg3 = XGBRegressor(objective ='reg:squarederror', scale_pos_weight = 5)
xgb_reg3.fit(X_train, y_train)

# Test xgb_reg
y_pred_xgb3 = xgb_reg3.predict(X_test)


mae_xgb3 = mean_absolute_error(y_test, y_pred_xgb3)
print('SRV MAE', mae_xgb3)

mse_xgb3 = mean_squared_error(y_test, y_pred_xgb3)
print('SRV MSE', mse_xgb3)

SRV MAE 0.41565080308949187
SRV MSE 0.35446751679789773


In [169]:
y_pred_xgb10 = np.rint(y_pred_xgb3)
y_test10 = np.rint(y_test)
print(classification_report(y_test10, y_pred_xgb10))

              precision    recall  f1-score   support

         0.0       0.75      0.67      0.71         9
         1.0       0.50      0.50      0.50         4
         2.0       0.00      0.00      0.00         3
         3.0       0.29      0.27      0.28        30
         4.0       0.77      0.82      0.79       103

    accuracy                           0.67       149
   macro avg       0.46      0.45      0.45       149
weighted avg       0.65      0.67      0.66       149



C:\Users\lukez\anaconda3\envs\ComputerVision\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\lukez\anaconda3\envs\ComputerVision\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\lukez\anaconda3\envs\ComputerVision\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

In [114]:
# Build a xgb regression model and train it on (X_train, y_train)
xgb_reg4 = XGBRegressor(objective ='reg:squarederror', scale_pos_weight = 20)
xgb_reg4.fit(X_train, y_train)

# Test xgb_reg
y_pred_xgb4 = xgb_reg4.predict(X_test)


mae_xgb4 = mean_absolute_error(y_test, y_pred_xgb4)
print('SRV MAE', mae_xgb4)

mse_xgb4 = mean_squared_error(y_test, y_pred_xgb4)
print('SRV MSE', mse_xgb4)

SRV MAE 0.3915466176183432
SRV MSE 0.33503755596515433


In [115]:
# Build a xgb regression model and train it on (X_train, y_train)
xgb_reg5 = XGBRegressor(objective ='reg:squarederror', scale_pos_weight = 25)
xgb_reg5.fit(X_train, y_train)

# Test xgb_reg
y_pred_xgb5 = xgb_reg5.predict(X_test)


mae_xgb5 = mean_absolute_error(y_test, y_pred_xgb5)
print('SRV MAE', mae_xgb5)

mse_xgb5 = mean_squared_error(y_test, y_pred_xgb5)
print('SRV MSE', mse_xgb5)

SRV MAE 0.39808934595710876
SRV MSE 0.33214975494364074


In [116]:
# Build a xgb regression model and train it on (X_train, y_train)
xgb_reg4 = XGBRegressor(objective ='reg:squarederror', scale_pos_weight = 30)
xgb_reg4.fit(X_train, y_train)

# Test xgb_reg
y_pred_xgb4 = xgb_reg4.predict(X_test)


mae_xgb4 = mean_absolute_error(y_test, y_pred_xgb4)
print('SRV MAE', mae_xgb4)

mse_xgb4 = mean_squared_error(y_test, y_pred_xgb4)
print('SRV MSE', mse_xgb4)

SRV MAE 0.38791967991539494
SRV MSE 0.32436966084099167


In [170]:
y_pred_xgb10 = np.rint(y_pred_xgb4)
y_test10 = np.rint(y_test)
print(classification_report(y_test10, y_pred_xgb10))

              precision    recall  f1-score   support

         0.0       1.00      0.22      0.36         9
         1.0       0.40      1.00      0.57         4
         2.0       0.00      0.00      0.00         3
         3.0       0.18      0.13      0.15        30
         4.0       0.76      0.84      0.80       103

    accuracy                           0.65       149
   macro avg       0.47      0.44      0.38       149
weighted avg       0.63      0.65      0.62       149



C:\Users\lukez\anaconda3\envs\ComputerVision\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\lukez\anaconda3\envs\ComputerVision\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\lukez\anaconda3\envs\ComputerVision\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

In [117]:
# Build a xgb regression model and train it on (X_train, y_train)
xgb_reg4 = XGBRegressor(objective ='reg:squarederror', scale_pos_weight = 35)
xgb_reg4.fit(X_train, y_train)

# Test xgb_reg
y_pred_xgb4 = xgb_reg4.predict(X_test)


mae_xgb4 = mean_absolute_error(y_test, y_pred_xgb4)
print('SRV MAE', mae_xgb4)

mse_xgb4 = mean_squared_error(y_test, y_pred_xgb4)
print('SRV MSE', mse_xgb4)

SRV MAE 0.4239492712404904
SRV MSE 0.3623154056816202


In [118]:
# Build a xgb regression model and train it on (X_train, y_train)
xgb_reg4 = XGBRegressor(objective ='reg:squarederror', scale_pos_weight = 40)
xgb_reg4.fit(X_train, y_train)

# Test xgb_reg
y_pred_xgb4 = xgb_reg4.predict(X_test)


mae_xgb4 = mean_absolute_error(y_test, y_pred_xgb4)
print('SRV MAE', mae_xgb4)

mse_xgb4 = mean_squared_error(y_test, y_pred_xgb4)
print('SRV MSE', mse_xgb4)

SRV MAE 0.4306287106889106
SRV MSE 0.3726664841874955


In [120]:
y_pred_xgb10 = np.rint(y_pred_xgb4)
y_test10 = np.rint(y_test)
print(classification_report(y_test10, y_pred_xgb10))
print(confusion_matrix(y_test10, y_pred_xgb10))

              precision    recall  f1-score   support

         0.0       1.00      0.22      0.36         9
         1.0       0.40      1.00      0.57         4
         2.0       0.00      0.00      0.00         3
         3.0       0.18      0.13      0.15        30
         4.0       0.76      0.84      0.80       103

    accuracy                           0.65       149
   macro avg       0.47      0.44      0.38       149
weighted avg       0.63      0.65      0.62       149

[[ 2  6  0  0  1]
 [ 0  4  0  0  0]
 [ 0  0  0  2  1]
 [ 0  0  0  4 26]
 [ 0  0  0 16 87]]


C:\Users\lukez\anaconda3\envs\ComputerVision\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\lukez\anaconda3\envs\ComputerVision\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\lukez\anaconda3\envs\ComputerVision\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

In [122]:
print(y_pred_xgb10)

[ 4.  4.  4.  3.  1.  4.  4.  4.  1.  4.  4.  3.  4.  4.  4.  4.  3.  4.
  4.  1.  4.  3.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.
  4.  1.  4.  4.  4.  4.  3.  4.  4.  4.  3.  1.  4.  4.  4.  4.  4.  3.
  3.  3.  4.  4.  3.  4.  4.  4.  4.  4.  4.  4.  4.  3.  4.  1.  4.  4.
  4.  4.  4.  4.  4. -0.  4.  0.  4.  4.  4.  4.  3.  4.  4.  4.  4.  4.
  4.  4.  3.  4.  4.  4.  4.  3.  4.  4.  4.  4.  4.  4.  3.  4.  4.  4.
  4.  4.  4.  4.  4.  3.  4.  1.  3.  4.  1.  4.  4.  4.  3.  4.  3.  4.
  4.  4.  4.  3.  3.  4.  1.  4.  4.  4.  4.  4.  4.  1.  4.  4.  4.  4.
  4.  4.  4.  3.  4.]


In [132]:
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

In [133]:
print(
'\nmR',recall_score(y_test10, y_pred_xgb10, average='micro'),'\nmP', precision_score(y_test10, y_pred_xgb10, average='micro'),
'\nMR',recall_score(y_test10, y_pred_xgb10, average='macro'),'\nMP', precision_score(y_test10, y_pred_xgb10, average='macro'),
'\nmF1',f1_score(y_test10, y_pred_xgb10,average='micro'),'\nMF1', f1_score(y_test10, y_pred_xgb10,average='macro'))


mR 0.6510067114093959 
mP 0.6510067114093959 
MR 0.4400431499460626 
MP 0.4676679841897233 
mF1 0.6510067114093959 
MF1 0.3774152453051536


C:\Users\lukez\anaconda3\envs\ComputerVision\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [137]:
print(labels)

[1, 1, 1, 1, 1, 1, 1, 4, 1, 0, 0, 0, 4, 4, 3, 2, 3, 3, 3, 0, 2, 4, 3, 3, 4, 2, 4, 4, 4, 4, 3, 3, 4, 3, 4, 3, 3, 2, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 2, 3, 4, 3, 3, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 0, 1, 1, 3, 4, 3, 4, 3, 4, 3, 3, 3, 2, 4, 4, 4, 3, 3, 3, 3, 4, 3, 3, 3, 4, 4, 4, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 4, 3, 3, 4, 4, 4, 4, 4, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 3, 3, 4, 4, 3, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 2, 3, 3, 3, 4, 3, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 0, 3, 3, 3, 2, 2, 0, 4, 4, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 

In [140]:
lablesNormilized = np.divide(labels, 4)

In [141]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(flattened_data, lablesNormilized, test_size=0.2)

In [163]:
X_test1.shape

(149, 62507)

In [ ]:
# Build a xgb regression model and train it on (X_train, y_train)
xgb_reg4 = XGBRegressor(objective ='reg:squarederror', scale_pos_weight = 40)
xgb_reg4.fit(X_train, y_train)

# Test xgb_reg
y_pred_xgb4 = xgb_reg4.predict(X_test)


mae_xgb4 = mean_absolute_error(y_test, y_pred_xgb4)
print('SRV MAE', mae_xgb4)

mse_xgb4 = mean_squared_error(y_test, y_pred_xgb4)
print('SRV MSE', mse_xgb4)